# Upload and download widgets

The jp_proxy_widget package includes helpers for uploading and downloading
files.  An upload moves a file from the "user workstation area" to the
"Jupyter server area".  A download moves a file from the server area to the
workstation area.  The widgets also allow the program to control file names
and transform or filter file content.


# Using the uploader widget: basic usage

The following widget uploads a file using the same name on the server
as the name from the workstation.

In [ ]:
from jp_proxy_widget import uploader

# Make an upload button which uploads a binary file to the same file name
upload = uploader.BinaryUploader("file to upload", to_filename=True)
upload.widget

In [ ]:
fn = upload.uploaded_filename
if fn:
    print ("This file was uploaded: " + fn)
else:
    print ("No file was uploaded yet.")

# Using the uploader widget: more advanced usage

The following demonstrates how to upload a file using the 
`jp_proxy_widget.uploader.BindaryUploader` widget.  The widget
has additional configuration to display information about the uploaded
file, to optionally change the file name for the upload, and to display
the uploaded image in HTML.

In [ ]:

from IPython.display import HTML, display

In [ ]:
# Upload a file and try to display it as an image
class UploadImage:
    
    name = None
    
    def __init__(self):
        # Create a widget which displays a button that uploads a binary file.
        self.uploader = uploader.BinaryUploader(
            html_title="upload image", 
            content_callback=self.content_callback)
        w = self.widget = self.uploader.widget
        # Add some Javascript functionality to specify the file name and display results
        w.js_init("""
        
        // Add a text input to change the name of the file uploaded.
        var name_div = $("<div/>").appendTo(element);
        $("<b>Upload to this file name: </b>").appendTo(name_div);
        var name_input = $('<input type="text"/>').appendTo(name_div);
        
        // Add an informational text area and call-in to update it.
        var info_area = $("<div>Please upload an image file</div>").appendTo(element);
        name_input.on("input", function(e) {change_name(name_input.val())});
        element.informational = function(text) {
            info_area.html("<div>" + text + "</div>");
        };
        
        // Add a call-in used to display the uploaded image.
        element.add_html = function(html) {
            $(html).appendTo(element);
        }
        
        """, change_name = self.change_name)
        
    def change_name(self, name):
        "Store the optional file name from the text input area."
        self.name = name.strip()
        
    def content_callback(self, widget, name, content):
        "Perform the upload and display the image uploaded."
        use_name = name
        if self.name:
            # use the name override if specified (non-white)
            use_name = self.name
        info = self.widget.element.informational
        info("saving " + repr((len(content), name, use_name)))
        # Save the file
        f = open(use_name, "wb")
        f.write(content)
        f.close()
        # Append HTML to display the image uploaded.
        img_html = '<div>Uploaded %s:</div><div><img src="%s"/></div>' % (use_name, use_name)
        self.widget.element.add_html(img_html)
        

In [ ]:
u = UploadImage()

In [ ]:
#u.widget.debugging_display()
u.uploader.show()

# Using the downloader widget: basic usage

The downloader operation must be executed by an existing proxy widget.

In [ ]:
from jp_proxy_widget import uploader, downloader
import jp_proxy_widget

# We use this widget to execute the download:
the_download_widget = jp_proxy_widget.JSProxyWidget()

# The filename and file content we want to download:
filename = "example_file.binary"
content = b"some binary content " + bytes(range(14,23)) + bytes(range(244,253))

# The download action:
def do_download(*ignore_arguments):
    downloader.saveAsBinary(the_download_widget, filename, content)
    
# Configure the widget with a button to trigger the download.
the_download_widget.js_init("""
    element.empty();
    button = $("<button>Download binary data</button>").appendTo(element);
    button.click(do_download);
""", do_download=do_download)

# show the widget
#the_download_widget.debugging_display()
the_download_widget

In [ ]:
the_download_widget.print_status()